<font size="6">Модели машинного обучения</font>

# Необходимость методов классического машинного обучения

<center><img src="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L03/out/ml_dl_learning_plateau.png" alt="alttext" width="600"/></center>

Существует ряд причин, почему важно знать и уметь применять методы классического машинного обучения:

* **Недостаточный объем данных для нейронных сетей.** При малом объеме данных методы классического машинного обучения показывают более высокую эффективность.
* **Время получения решения задачи.** Скорость экспериментов и получения хорошего бейзлайна выше, чем у решения с помощью нейронных сетей.
*  **Интерпретируемость предсказаний.** Из методов классического машинного обучения легче получить важность каждого отдельно взятого признака.
* **Комбинирование подходов.** В современных задачах часто используются комбинации нейронных сетей и алгоритмов классического машинного обучения. Например, можно использовать нейронные сети в качестве генераторов признаков для методов классического обучения.

Зачастую ваши задачи могут быть полностью и частично сведены к использованию методов машинного обучения. На пришлой лекции мы познакомились с основами, сегодня мы познакомимся с более мощными моделями.

Далее познакомимся с логистической регрессией, но для лучшего понимания  вначале объясним общий подход на примере линейной регрессии.

# Линейная регрессия


**Регрессия** — это одна из трех базовых задач машинного обучения (классификация, регрессия, кластеризация).

В задаче **регрессии** мы используем входные **признаки**, чтобы предсказать **целевые значения**. **Линейная регрессия** сводится к тому, чтобы провести “**линию наилучшего соответствия**” через набор точек данных.


##  Модель и ее параметры

Предположим, у нас есть набор точек $\{(x_i, y_i)\}$.

Цель линейной регрессии — **поиск линии, которая наилучшим образом соответствует заданным точкам**. Напомним, что общее уравнение прямой:

$$\large f(x) = w⋅x + b,$$

где $w$ — характеризует наклон линии (в будущем мы будем называть значения $w$ весом, weight) а $b$ — её сдвиг по оси $y$ (bias). Таким образом, решение линейной регрессии определяет значения для $w$ и $b$ так, что $f(x)$ приближается как можно ближе к $y(x)$. Здесь $w$ и $b$ — **параметры модели**.

Отобразим на графике случайные точки, расположенные в окрестности $y(x) = 6⋅x + 2$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


np.random.seed(42)
x = np.random.rand(100, 1)
y = 2 + 6 * x + (np.random.rand(100, 1) - 0.5)

plt.figure(figsize=(5, 3))
plt.scatter(x, y, s=10)
plt.xlabel("x")
plt.ylabel("y")
plt.show()

Предположим, что нам неизвестны параметры наклона и сдвига $w$ и $b$. Для их определения мы бы могли рассмотреть все возможные прямые вида $f(x) = w⋅x + b$ и выбрать среди семейства прямых такую, которая лучше всего приближает имеющиеся данные:

In [ ]:
plt.figure(figsize=(5, 3))
plt.scatter(x, y, s=10)
for w in [6.1, 5.8]:
    for b in [1.9, 2.3]:
        y_predicted = w * x + b
        plt.plot(x, y_predicted, color="r", alpha=0.3)
plt.plot(x, 2 + 6 * x, color="g")
plt.xlabel("x")
plt.ylabel("y")
plt.show()

## Функция потерь

Как выбрать параметры?

**Функция потерь** позволяет вычислить меру количества ошибок. Для задачи **регрессии** такой мерой может быть **расстояние** между предсказанным значением $f(x)$ и его фактическим значением. Распространенной функцией потерь является **средняя квадратичная ошибка** (MSE). Чтобы вычислить MSE, мы просто берем все значения ошибок, считаем квадраты их длин и усредняем.

То есть мы определяем ошибку модели на одном объекте как квадрат расстояния между предсказанием и истинным значением, а общая функция потерь будет задана выражением:

$$l_i =|y_i - f(x_i)| $$

$$ \text{Loss} = \sum l_i^2 = \frac{1}{N} \sum (y_i - f(x_i))^2$$

Для прямой с параметрами $w=4$, $b = 2$ и $w=3$, $b = 2$ (верные значения):

In [ ]:
def plot_delta_line(ax, x, y, w, b, color="r"):
    y_predicted = w * x + b
    # line
    ax.plot(x, y_predicted, color=color, alpha=0.5, label=f"f(x)={w}x+{b}")
    # delta
    for x_i, y_i, f_x in zip(x, y, y_predicted):
        ax.vlines(x=x_i, ymin=min(f_x, y_i), ymax=max(f_x, y_i), ls="--", alpha=0.3)
    # MSE
    loss = np.sum((y - (w * x + b)) ** 2) / (len(x))
    ax.set_title(f"MSE = {loss:.3f}")
    ax.legend()


fig, axs = plt.subplots(1, 2, figsize=(11, 4))

# plot x_i y_i (dots)
for ax in axs:
    ax.scatter(x, y, s=10)
    ax.set_xlim([0, 0.8])
    ax.set_ylim([2, 6])
    ax.set_xlabel("x")
    ax.set_ylabel("y")

plot_delta_line(axs[0], x, y, w=5, b=2, color="r")
plot_delta_line(axs[1], x, y, w=6, b=2, color="g")

plt.show()

Задача **поиска оптимальных параметров** модели сводится к задаче **поиска минимума функции потерь**.

# Метод градиентного спуска

Задача поиска оптимальных параметров модели сводится к задаче **поиска минимума функции потерь**.

## Точки минимума и максимума функции

Как найти минимум функции в простом случае?

- Найти производную функции.
- Найти значения, при которых производная равна нулю.
- Определить знаки производной. Когда функция возрастает, то производная положительна. Когда функция убывает, то производная отрицательна.
- Определить точки минимума и максимума. Если функция возрастала и в определенной точке начала убывать, то это точка максимума. Если функция убывала и в некоторой точке начала возрастать, то это точка минимума.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L01/function.png" width="700" ></center>

$$\text{Таблица производных:}$$
\begin{array}{|с|c|c|} \hline
 f(x) \text{ (функция)} & f'(x)  \text{ (производная)}\\ \hline
с \text{ (константа)} & 0 \\ \hline
cx & c \\ \hline
x^c & cx^{c-1} \\ \hline
\end{array}

Рассмотрим на примере функции $f(x)=x^2 + x + 3$.

Она зависит от одной переменной $x$.

Найдем производную $f'(x^2 - x + 3)$.
- $f'(x^2 - x + 3) = 2x - 1$
- $f'(x^2 - x + 3) = 0$ при $x =0.5$
-  $f'(x^2 - x + 3) < 0 $ при $x < 0.5$, $f'(x^2 - x + 3) > 0 $ при $x > 0.5$
- $f(x)$ убывает при $x < 0.5$, $f(x)$ возрастает при $x > 0.5$
- $x = 0.5$ — точка минимума

In [ ]:
x = np.linspace(-2, 3, 100)
y = x**2 - x + 3
fig, ax = plt.subplots()
fig.suptitle("$x^2 - x + 3$")
ax.plot(x, y)
ax.set(xticks = np.arange(-2, 3.5, step=0.5))
ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
ax.plot(0.5,2.75, marker='.')
fig.show()

## Ландшафт функции потерь

Посмотрим, как будет выглядеть ландшафт функции потерь для нашей задачи.

In [ ]:
w = np.arange(-10, 30, 1)
b = np.arange(-10, 10, 1)

x = np.random.rand(100, 1)
y = 2 + 6 * x + (np.random.rand(100, 1) - 0.5)

ww, bb = np.meshgrid(w, b)

loss = np.zeros_like(ww)
for i in range(ww.shape[0]):
    for j in range(ww.shape[1]):
        loss[i, j] = np.sum((y - (ww[i, j] * x + bb[i, j])) ** 2) / (len(x))

def show_3d(xx, yy, zz, fig, title = "MSE", zlim = (0,20), alpha=0.5):
    ax = fig.add_subplot(121, projection="3d")
    surf = ax.plot_surface(xx, yy, zz, cmap=plt.cm.RdYlGn_r, alpha=alpha)

    ax.contourf(xx, yy, zz, zdir="zz", offset=-1, cmap="RdYlGn_r", alpha=alpha)
    ax.set_zlim(zlim)

    ax.set_xlabel("b")
    ax.set_ylabel("w")
    ax.set_title(title)
    fig.colorbar(surf, location="left")

fig = plt.figure(figsize=(15, 5))
show_3d(ww, bb, loss, fig)

Необходимым (но недостаточным) условием локального минимума дифференцируемой функции является равенство нулю частных производных:

$$	\begin{equation*}
 \begin{cases}
   \displaystyle\frac{\partial \text{Loss}}{\partial w}=0,
   \\
   \displaystyle\frac{\partial \text{Loss}}{\partial b}=0.
 \end{cases}
\end{equation*} $$

Т.к. MSE для линейной регрессии — полином второй степени относительно $w$ и $b$, а полином второй степени не может иметь больше одного экстремума, то локальный минимум будет глобальным.

## Частная производная и градиент функции

Если функция зависит от нескольких переменных, можно говорить о частной производной, когда все остальные переменные, кроме интересующей нас, становятся константами. Производная функции $f$ от переменной $x_1$: $\large\frac{\partial f}{\partial x_1}$.

**Градиент** в математическом анализе — это вектор, указывающий на направление максимального роста функции в заданной точке. Вектор-градиент состоит из частных производных функции от каждой из ее переменных $(x_{1},...,x_{n})$:

$$ \nabla f(\vec{x}) = \begin{bmatrix}
\displaystyle\frac{\partial f}{\partial x_1}\\
\displaystyle\frac{\partial f}{\partial x_2}\\
...\\
\displaystyle\frac{\partial f}{\partial x_n}\\
\end{bmatrix}$$

Рассмотрим функцию $\large f(x, y) = \sin(x\cdot y)$ и посчитаем градиент . Для этого воспользуемся [**таблицей производных** 📚[wiki]](https://ru.wikipedia.org/wiki/%D0%A2%D0%B0%D0%B1%D0%BB%D0%B8%D1%86%D0%B0_%D0%BF%D1%80%D0%BE%D0%B8%D0%B7%D0%B2%D0%BE%D0%B4%D0%BD%D1%8B%D1%85) и правилом вычисления [**производной сложной функции** 📚[wiki]](https://ru.wikipedia.org/wiki/%D0%94%D0%B8%D1%84%D1%84%D0%B5%D1%80%D0%B5%D0%BD%D1%86%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_%D1%81%D0%BB%D0%BE%D0%B6%D0%BD%D0%BE%D0%B9_%D1%84%D1%83%D0%BD%D0%BA%D1%86%D0%B8%D0%B8) (Chain-rule):
$$\frac {\partial f} {\partial x} = \frac {\partial f} {\partial t} \cdot \frac {\partial t} {\partial x}$$

Это правило очень нам пригодится в будущем.

$$\nabla f(x, y)=\begin{bmatrix}
\displaystyle\frac{\partial f}{\partial x}\\
\displaystyle\frac{\partial f}{\partial y}\\
\end{bmatrix}
=\begin{bmatrix}
\displaystyle\frac{\partial\sin(xy)}{\partial(xy)}\cdot\frac{\partial(xy)}{\partial x}\\
\displaystyle\frac{\partial\sin(xy)}{\partial(xy)}\cdot\frac{\partial(xy)}{\partial y}\\
\end{bmatrix}
= \begin{bmatrix}
\cos(xy)\cdot y\\
\cos(xy)\cdot x\\
\end{bmatrix}
$$


Для минимизации ошибки нужен антиградиент, который показывает направление скорейшего убывания функции.

In [ ]:
f = lambda x, y: np.sin(x * y)

x = np.linspace(0, 4, 1000)
y = np.linspace(0, 4, 1000)
xx, yy = np.meshgrid(x, y)
zz = f(xx, yy)

In [ ]:
gradf = lambda x, y: (np.cos(x * y) * y, np.cos(x * y) * x)

xsmall = np.linspace(0, 4, 15)
ysmall = np.linspace(0, 4, 15)
xxsmall, yysmall = np.meshgrid(xsmall, ysmall)
gradx, grady = gradf(xxsmall, yysmall)

Так как **значение градиента в точке** — это вектор, мы можем говорить о его **величине** и **направлении**. Визуализируем наши расчеты: посмотрим на ландшафт функции $f(x, y)$ и направления градиентов.


In [ ]:
fig = plt.figure(figsize=(15, 5))
show_3d(xx, yy, zz, fig, title = "sin(xy)", zlim = (-2,2), alpha = 1)

ax = fig.add_subplot(122)
ax.imshow(
    zz,
    extent=(np.min(x), np.max(x), np.min(y), np.max(y)),
    cmap="RdYlGn_r",
    origin="lower",
)
ax.set_xlabel("x")
ax.set_ylabel("y")

ax.quiver(xxsmall, yysmall, gradx, grady)
plt.show()

На рисунке выше значения градиента в точке обозначены чёрными стрелочками. Можно заметить, что длина стрелок в  районе максимальных и минимальных значений функции **почти нулевая**, стрелки направлены в направлении возрастания значения функции и наиболее длинные стрелки находятся в области наиболее резкого изменения значений функции.


Это проявление **свойств градиента**:
* Направление $\frac{\nabla f}{||\nabla f||}$ — сообщает нам направление максимального роста функции.

*  Величина $||\nabla f||$ — характеризует мгновенную скорость изменения значений функции.

## Градиентный спуск и скорость обучения

**Градиентный спуск** — это алгоритм для итеративного, шаг за шагом, перемещещния в поиске минимума. Мы не напрямую ищем точку, где производная равна нулю, а постепенно передвигаемся вниз по функции. Градиентный спуск выбирает случайную точку, находит направление самого быстрого убывания функции и двигается до ближайшего минимума вдоль этого направления.

Важно настроить размер шага или **скорость обучения** $η$ — некоторый коэффициент, как правило, небольшой, который не позволяет нам двигаться слишком быстро. У нас есть точка, в которую мы хотим попасть. Если мы сделаем слишком большой шаг, то мы ее перескочим (график справа), поэтому надо подобрать шаг, который не позволит ее перескочить, но в то же время такой, чтобы тот же процесс не шел слишком медленно (как на графике слева).

<img src ="https://edunet.kea.su/repo/EduNet_NLP-content/L01/out/learning_rate_optimal_value.png">

Если на каком-то этапе разность между старой точкой (до шага) и новой снижается ниже предела, считается, что минимум найден, алгоритм завершен.

Вспомним, что функция потерь $\mathcal{L}(y, \hat{y})$ зависит от нескольких переменных: весов $\overline{w} = (w_1, \cdots, w_m)$ и сдвига $b$.

Следовательно, мы будем использовать несколько частных производных:

- частные производные функции потерь от весов $w_1,\cdots,w_m$: $\large\frac{\partial \mathcal{L}(y, \hat{y})}{\partial w_1}, \cdots, \frac{\partial \mathcal{L}(y, \hat{y})}{\partial w_m}$
- частная производная функции потерь от сдвига $b$: $\large\frac{\partial \mathcal{L}(y, \hat{y})}{\partial b}$

После подсчета частных производных нам необходимо обновить значения весов и сдвига.

$$w_i = w_i-η\frac{\partial \mathcal{L}(y, \hat{y})}{\partial w_i}$$

$$b = b-η\frac{\partial \mathcal{L}(y, \hat{y})}{\partial b}$$

Реализуем градиентный спуск и решим с его помощью нашу задачу с линейной регрессией.

In [ ]:
from sklearn.model_selection import train_test_split

x = np.random.rand(250, 1)
y = 2 + 6 * x + (np.random.rand(250, 1) - 0.5)

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42
)

print (x_train.shape, x_test.shape)
print (y_train.shape, y_test.shape)

**Запись смещения**

Для компактной записи смещения можно считать его таким же весом, который домножается на 1.

Обозначим вектор-столбец из настраиваемых параметров:
$$\vec w = \begin{bmatrix}
b \\ w \\
\end{bmatrix}$$

К матрице (в нашем случае был только один признак, поэтому у нас будет вектор-столбец) признаков слева "дорисуем" столбец единиц:
$$X = \begin{bmatrix}
1 & X \\
\end{bmatrix} =
\begin{bmatrix}
1 & 2.7 \\
1 & 3.3 \\
... & ...\\
1 & 9.2 \\
\end{bmatrix}$$

**Предупреждение:** добавлять столбец единиц нужно, только если вы сами пишете модель. **Если вы пользуетесь готовыми моделями, в этом нет необходимости.**


В общем случае:

$$\large \vec y = b + w_1\cdot x_1 + w_2\cdot x_2 + w_3\cdot x_3 + w_4\cdot x_4 + w_5\cdot x_5 + ... + w_n\cdot x_n = X\vec w $$  

In [ ]:
x_train = np.hstack((np.ones((x_train.shape[0], 1)), x_train))
x_test = np.hstack((np.ones((x_test.shape[0], 1)), x_test))

print (x_train.shape, x_test.shape)

Реализация градиентного спуска. В будущем это будет спрятано "внутри" моделей.

In [ ]:
from sklearn.metrics import mean_squared_error

def gradient(x, y, w):
    """Gradient of mean squared error."""
    return 2 * (x.T @ (x @ w) - x.T @ y) / len(x)


def gradient_descent(x_train, y_train, x_test, y_test, w, alpha, iteration=10):
    """Gradient descent for optimizing slope in simple linear regression"""
    # history
    ws = np.zeros((iteration + 1, 2))
    ws[0] = w[:, 0]
    mse_train = [mean_squared_error(y_train, x_train @ w)]
    dmse_train = []
    mse_test = [mean_squared_error(y_test, x_test @ w)]
    prediction = {(w[0][0], w[1][0]): x_train @ w}

    print(
        f"Iteration 0: b = {w[0][0]:.2f}, w = {w[1][0]:.2f}, "
        f"Loss_train = {mse_train[0]:.2f}, "
        f"Loss_test = {mse_test[0]:.2f}."
    )

    for i in range(iteration):
        # adjust w based on gradient * learning rate
        grad = gradient(x_train, y_train, w)
        w -= alpha * grad  # adjust w based on gradient * learning rate
        # history
        ws[i + 1] = w[:, 0]
        mse_train.append(mean_squared_error(y_train, x_train @ w))
        dmse_train.append(grad)
        mse_test.append(mean_squared_error(y_test, x_test @ w))
        prediction[(w[0][0], w[1][0])] = x_train @ w

        print(
            f"Iteration {i+1}: b = {w[0][0]:.2f}, w = {w[1][0]:.2f}, "
            f"Loss_train = {mse_train[i]:.2f}, "
            f"Loss_test = {mse_test[i]:3.2f}."
        )
    return ws, prediction, mse_train, dmse_train, mse_test

Обучим нашу модель, задав начальные значения $b$ и $w$, а также скорость обучения:

In [ ]:
w = np.array([[-5.0], [-3.0]])
ws, prediction, mse_train, dmse_train, mse_test = gradient_descent(
    x_train,
    y_train,
    x_test,
    y_test,
    w,
    alpha = 0.001,
)

Визуализируем процесс:

In [ ]:
def plot_mse(mse_train, mse_test):
    plt.figure(figsize=(10, 4))
    plt.title("Learning curve")
    plt.plot(mse_train, label="train")
    plt.plot(mse_test, label="test")
    plt.legend()

    plt.xlabel("iterations", fontsize=12)
    plt.ylabel("MSE Loss", fontsize=12)

    plt.grid(True)
    plt.show()

In [ ]:
plot_mse(mse_train, mse_test)

Видим, что ошибка постепенно падает, но до сходимости далеко. Также отмечаем, что **ошибка на тесте меньше ошибки на обучении**. **Что-то явно не так.** Так бывает при утечке данных (об утечке данных вы подробнее узнаете в следующих лекциях), но в данном случае, test выборка просто слишком мала, чтобы отражать генеральную совокупность.

Попробуем увеличить скорость обучения.

In [ ]:
w = np.array([[-5.0], [-3.0]])
ws, prediction, mse_train, dmse_train, mse_test = gradient_descent(
    x_train,
    y_train,
    x_test,
    y_test,
    w,
    alpha = 1,
)

In [ ]:
plot_mse(mse_train, mse_test)

Слишком большая скорость, модель не сошлась: ошибка растёт.

In [ ]:
w = np.array([[-5.0], [-3.0]])
ws, prediction, mse_train, dmse_train, mse_test = gradient_descent(
    x_train,
    y_train,
    x_test,
    y_test,
    w,
    alpha = 0.2,
)

В дальнейших лекциях вы познакомитесь с различными модификациями метода градиентного спуска и узнаете больше о выборе скорости обучения, а пока ориентируйтесь на кривые обучения.

## Необходимость нормализации

Выше мы не дошли до оптимальной прямой  $y=6x+2$. При этом график Loss выглядит неплохо:

In [ ]:
plot_mse(mse_train, mse_test)

Такое поведение связано с ландшафтом функции потерь: значение ошибки по оси $b$ изменяется намного медленнее, чем по оси $w$.

Скачаем **код для интерактивной визуализации**. Он нужен только для объяснения и **не пригодится вам в работе**. Его разбирать мы не будем. Eсли интересно, можно изучить самостоятельно.

In [ ]:
# @title *Code for interactive visual
# source: https://github.com/TomasBeuzen/deep-learning-with-pytorch

!wget -qN https://edunet.kea.su/repo/EduNet-web_dependencies/dev-2.1/L02/interactive_visualization.py

In [ ]:
from interactive_visualization import plot_grid_search_2d

intercepts = np.arange(-7.5, 12.5, 0.1)  # b
slopes = np.arange(-5, 5, 0.1)  # w
plot_grid_search_2d(x_train[:, 1], y_train, slopes, intercepts)

Поэтому основное изменение значений происходит вдоль оси $w$, а $b$ меняется слабо (значение $b$ далеко от ожидаемого).

In [ ]:
from interactive_visualization import plot_gradient_descent_2d

plot_gradient_descent_2d(
    x_train[:, 1],
    y_train[:, 0],
    ws,
    slopes,
    intercepts,
)

Чтобы исправить ситуацию, применим `StandardScaler`:

Идея **`StandardScaler`** заключается в том, что он преобразует данные таким образом, что распределение будет иметь среднее значение $0$ и стандартное отклонение $1$. Большинство значений будет находиться в диапазоне от $-1$ до $1$. Это стандартная трансформация, и она применима во многих ситуациях.

$$\large z_i=\frac{X_i-u}{s},$$

где $u$ — среднее значение (или $0$ при `with_mean=False`), $s$ — стандартное отклонение (или $0$ при `with_std=False`).

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

x_train_scaled = scaler.fit_transform(np.expand_dims(x_train[:, 1], axis=1)).flatten()
x_test_scaled = scaler.transform(np.expand_dims(x_test[:, 1], axis=1)).flatten()

Отметьте ключевой момент. Среднее значение и дисперсия вычисляются **только по тренировочной выборке**.

Как правило, решения, которые переводят текст в числовое представление, обеспечивают тот или иной вариант нормировки, но это стоит проверять. Существуют и другие алгоритмы нормировки, об этом подробнее в  [[wiki] 📚 L02 MSU.AI. Линейные модели](https://msu.ai/l02_linear_models)

In [ ]:
intercepts = np.arange(-15, 25, 0.2)  # b
slopes = np.arange(-20, 20, 0.2)  # w

plot_grid_search_2d(x_train_scaled, y_train, slopes, intercepts)

Теперь ландшафт функции потерь симметричен, обе переменных становятся во время обучения равнозначны.

In [ ]:
x_train_scaled = np.hstack(
    (np.ones((len(x_train_scaled), 1)), np.expand_dims(x_train_scaled, axis=1)),
)

x_test_scaled = np.hstack(
    (np.ones((len(x_test_scaled), 1)), np.expand_dims(x_test_scaled, axis=1)),
)

Т.к. диапазоны $x$ изменились, значения $w$ и $b$ тоже изменятся.


In [ ]:
w = np.array([[57.0], [33.0]])
ws, prediction, mse_train, dmse_train, mse_test = gradient_descent(
    x_train_scaled, y_train, x_test_scaled, y_test, w, 0.35, iteration=10
)

In [ ]:
plot_mse(mse_train, mse_test)

Проверим, что после нормализации мы сходимся к $y = 6x + 2$.  Для этого используем данные о матожидании и дисперсии из `StandardScaler`.

In [ ]:
b = ws[-1][0] - ws[-1][1] * scaler.mean_ / (scaler.var_) ** 0.5
w = ws[-1][1] / (scaler.var_) ** 0.5

print(f"y = {w[0]:.2f}x + {b[0]:.2f}")

По визуализации видно, что $w$ и $b$ изменяются во время обучения.

In [ ]:
plot_gradient_descent_2d(
    x_train_scaled[:, 1],
    y_train[:, 0],
    ws,
    slopes,
    intercepts,
)

## Cтохастический градиентный спуск

[[blog] ✏️ Пост о стохастическом градиентом спуске](https://www.tomasbeuzen.com/deep-learning-with-pytorch/chapters/chapter2_stochastic-gradient-descent.html)

До этого мы обучали модель, рассчитывая градиент по **всей train выборке**. Это не всегда возможно:
- данных может быть слишком много, чтобы загрузить их в память одновременно и рассчитать градиент,
- мы можем хотеть дообучать модель на свежепришедших данных, которых может быть немного.


Поэтому появляется идея **стохастического градиентного спуска**: мы можем делать шаг обучения, рассчитывая градиент не по всей выборке (**batch**), а по нескольким случайно выбранным объектам (**mini-batch**) или даже по одному случайно выбранному объекту (**stochastic**).

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L02/out/define_size_of_batch.png" width="500"></center>

Можно [показать 📚[book]](https://academy.yandex.ru/handbook/ml/article/shodimost-sgd), что **стохастический** (с размером $\text{batch}=1$) **градиентный спуск сходится к минимуму (глобальному или локальному) функции потерь** с конечной точностью. Важным условием является **стохастичность** (случайность). Если мы будем использовать одну и ту же последовательность выборок, это приведет к накоплению ошибки и смещению результата. Поэтому батчи на обучении формируются из различных объектов, каждый раз разные.

Имеет смысл использовать **максимальный размер mini-batch** для **ускорения расчетов**, .

В ходе работы с большими языковыми моделями вы увидите, что практически ничего не помещается в память, ни модель, ни данные, и мы будем знакомиться с различными трюками для обхода этих ограничений.

# Логистическая регрессия

Мы научились **решать задачу регрессии** множества переменных $x_1,x_2,...,x_n$ (признаков):

$$f(x) = w_1x_1 + w_2x_2 + ... + w_nx_n, $$

в которой $f$ может принимать значения в любом диапазоне. Чтобы перейти к **задаче бинарной классификации**, т.е. получению вероятность отнесения объекта к классу 0 или 1, нужно привести $f(x)$ в диапазон от 0 до 1.

Если классов более двух, задачу можно свести  к бинарной, если для каждого класса формулировать её как "один против всех".

## Переход к вероятности

В диапазон $[0,1]$ можно перевести с  помощью сигмоиды. Если получившееся значение больше 0.5, то объект относится к классу 1 (положительному), иначе — к классу 0 (отрицательному).

$$ σ(f(x))=\frac{1}{1+e^{-f(x)}}$$

Запишем формулу сигмоиды, используя методы из библиотеки NumPy:

- `np.exp(x)`  для подсчета $e^x$
- `np.arange()` для указания диапазона возможных значений

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math

x = np.arange(-10, 10, 0.5)
z = 1/(1 + np.exp(-x))

plt.plot(x, z)
plt.xlabel("x")
plt.ylabel("Sigmoid(X)")

plt.show()

Для многоклассовой классификации используется функция активации softmax.  Вероятность $i$-го класса при наличии $K$ классов рассчитывается следующим образом:

$$\text{softmax}(f_{i}) = \frac{e^{f_i}}{\sum_{j=1}^K e^{f_j}}$$
$$i=1,...,K$$

Для каждого объекта выбирается класс с наибольшей вероятностью.

Запишем формулу для функции софтмакс. Для подсчета суммы используем метод `np.sum()`.

In [ ]:
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x))

x = np.array([2.6, 3.2, 0.5])
print('Softmax:', softmax(x))
print('Sum:', np.sum(softmax(x)))

### Функция потерь

Мы бы хотели такую функцию, которая очень сильно штрафует за большие отступления от истинного ответа и не сильно за небольшие ошибки.

<center><img src ="https://i.ibb.co/FKsSgWL/cross-entropy.png" width="500" ></center>

Этим требованиям отвечает функция кросс-энтропии ($L_{CE}$ — *Cross Entropy Loss*):

$$L_{CE}(\hat y,y) = -\sum^{K}_{i=1}y_i \cdot log(\hat y_i),$$

где $i$ — номер класса, $y$ — истинный ответ, $\hat y$ — предсказанный ответ. Если истинный ответ 0, а предсказывается 1 — loss уходит в бесконечность. Однако, чем ближе мы приближаемся к 0 — тем существенно меньше штраф. В принципе, мы могли бы изобрести и другую функцию.

В случае бинарной классификации имеет $K=2$, $y=0$ или $y=1$ ($L_{BCE}$ — *Binary Cross Entropy Loss*):

$$L_{BCE}(\hat y,y) = -(y \cdot log(\hat y)+(1-y) \cdot log(1-\hat y))$$


В случае многоклассовой классификации $K>2$. Истинный ответ $y$ — вектор длины $K$, где элемент вектора $y_c=1$, если $c$ — истинный класс, остальные элементы равны $0$.

Например, $K=3$ (классы $0,1,2$). Объект относится к классу $2$. Тогда $y = (0, 0, 1)$, $c=2$, $y_2 = 1$.

Модель предсказывает вектор $\hat y$ длины $K$. Функция кросс-энтропии имеет вид:

$$L_{CE}(\hat y,y) = -\log \hat y_c$$

## Распознавание эмоций

Рассмотрим задачу многоклассовой классификации на примере распознавания эмоций.

Распознавание эмоций ≠ анализ тональности. Анализ тональности — выявление оценки авторов по отношению к объектам, речь о которых идёт в тексте. Она может быть позитивной, негативной или нейтральной. Эмоции могут включать не только радость или грусть, но ужас, гнев, удивление, отвращение и т.п.

Чаще всего при обработке текстов стоит задача распознать 6 базовых эмоций: счастье (happiness), печаль (sadness), страх (fear), гнев (anger), отвращение (disgust), удивление (surprise). Данная классификация введена Полом Экманом в книге [[book] 📚 Basic emotions](https://www.paulekman.com/wp-content/uploads/2013/07/Basic-Emotions.pdf). Утверждается, что люди всех культур испытывают их и могут распознавать в других людях. Для каждой базовой эмоции есть соответствующее, безошибочно опознаваемое выражение лица.

<center><img src ="https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/L01/basic_emotions.png" width="1100" ></center>

### Загрузка и подготовка данных

Будем использовать русскоязычный набор данных, представленный в работе [[paper] 🎓 Emotion classification in Russian: feature engineering and analysis](https://link.springer.com/chapter/10.1007/978-3-030-72610-2_10).

Он размечен по **5 классам эмоций**:
- радость (joy),
- печаль (sadness),
- злость (anger),
- неуверенность (uncertainty),
- нейтральность (neutrality).

Из классификации Экмана удалена категория отвращения из-за отсутствия соответствующих данных. Категории страха и удивления  объединены в одну категорию неопределенности из-за сходства способов, которыми они выражаются.

Загрузим набор данных.

In [ ]:
!wget -q https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/datasets/Emotion_Classification_Russian.csv

In [ ]:
import pandas as pd
emo = pd.read_csv('Emotion_Classification_Russian.csv')
emo.head()

<center><img src ="https://i.ibb.co/Jr5SprX/cleandata.png" width="700" ></center>

Вначале всех текстовых задач идёт очистка данных от "мусора": номеро встраниц и названий журналов, преобразование или удаление таблиц и картинок и прочее приведение текста в "чистый" вид.


Помимо метки класса (label) и текста (text), присутствует столбец с лемматизированными предложениями (lemmatized). Лемматизация — процесс приведения словоформы к лемме, то есть её нормальной (словарной) форме.

<center><img src ="https://i.ibb.co/wB19yJX/lemma.webp" width="500" ></center>

В русском языке это следующие морфологические формы:

- для существительных — именительный падеж, единственное число
  - кошками → кошка;
- для прилагательных — именительный падеж, единственное число, мужской род
  - красивых → красивый;
- для глаголов, причастий, деепричастий — глагол в инфинитиве (неопределённой форме) несовершенного вида
  - бежал → бежать.

Лемматизация необходима для языков с богатой морфологией, чтобы не расширять размер словаря за счет слов, у которых одинаковый смысл, но разные морфологические формы.

В рассматриваемом наборе данных лемматизация проведена с помощью морфологического анализатора [[git] 🐾 RNNmorph](https://github.com/IlyaGusev/rnnmorph). Он использует реккурентную нейронную сеть для определения части речи и морфологических признаков слова с учетом контекста.

Рассмотрим пример его применения.

In [ ]:
!pip install -q git+https://github.com/IlyaGusev/rnnmorph

In [ ]:
from rnnmorph.predictor import RNNMorphPredictor
predictor = RNNMorphPredictor(language="ru")

forms = predictor.predict(["мама", "поймала", "мышь"])
print(f'Часть речи слова "мама": {forms[0].pos}')
print(f'Лемма слова "поймала": {forms[1].normal_form}')
print(f'Морфологический анализ слова "мышь": {forms[2].tag}')

Для удобства заменим словесные обозначения классов в датасете на числовые. Метки присваиваются в алфавитном порядке:
- **a**nger → 0
- **j**oy → 1
- **n**eutrality →  2
- **s**adness → 3
- **u**ncertainty → 4

In [ ]:
emo['num_label'] = emo['label'].astype('category').cat.codes
emo.head()

Узнаем количество данных в датасете и их распределение по классам эмоций.

In [ ]:
emo.shape

In [ ]:
import matplotlib.pyplot as plt
plt.pie(emo['label'].value_counts(), labels=emo['label'].unique(), autopct='%.1f%%')
plt.show()

Проведем уже знакомые операции для подготовки данных:
- запишем в отдельные переменные лемматизированные тексты `X` и метки классов `y`;
- разделим данные на обучающую и тестовую выборку;
- осуществим векторизацию обучающей и тестовой выборок.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

X, y = emo['lemmatized'], emo['num_label']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify = y)

vect = TfidfVectorizer(min_df=0.0002, max_df=0.2)
X_train_vect = vect.fit_transform(X_train)
X_test_vect = vect.transform(X_test)
X_train_vect, X_test_vect

### Обучение и анализ модели

Обучим модель логистической регрессии (при дефолтном параметре `max_iter=100` модель не сходится; чтобы добиться сходимости алгоритма, установим большее количество итераций).

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(random_state=42, max_iter=300) # model initialization
logreg.fit(X_train_vect, y_train) # trainig
y_logreg = logreg.predict(X_test_vect) # predicting labels
y_logreg

Результаты классификации можно отразить в виде матрицы ошибок.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

def show_confusion_matrix(confusion_matrix):
  hmap = sns.heatmap(confusion_matrix, annot=True, fmt="d", cmap="Greens")
  hmap.yaxis.set_ticklabels(hmap.yaxis.get_ticklabels(), rotation=0, ha='right')
  hmap.xaxis.set_ticklabels(hmap.xaxis.get_ticklabels(), rotation=30, ha='right')
  plt.ylabel('True emotion')
  plt.xlabel('Predicted emotion')

class_names = ['anger', 'joy', 'neutrality', 'sadness', 'uncertainty']
cm = confusion_matrix(y_test, y_logreg)
df_cm = pd.DataFrame(cm, index=class_names, columns=class_names)
show_confusion_matrix(df_cm)

Метрики для каждого класса, а также усредненные метрики представлены в отчете о классификации.

In [ ]:
from sklearn.metrics import classification_report
target_names = ['anger', 'joy', 'neutrality', 'sadness', 'uncertainty']
print(classification_report(y_test, y_logreg, target_names=target_names))

Для каждого класса были посчитаны свои коэффициенты регрессии. Они представляют матрицу $K \times n$, где $K$ — количество классов, $n$ — количество признаков (слов).

In [ ]:
coefficient_matrix = logreg.coef_
#print(coefficient_matrix)
print('Размер матрицы: ', coefficient_matrix.shape)

 Каждый из коэффициентов регрессии описывает размер вклада соответствующего признака.
 - Положительный коэффициент — признак повышает вероятность принадлежности к классу, отрицательный коэффициент — признак уменьшает вероятность.
 - Большой коэффициент — признак существенно влияет на вероятность принадлежности к классу, почти нулевой коэффициент — признак имеет небольшое влияние на вероятность результата.

Выведем признаки с самыми большими коэффициентами, иными словами, получим наиболее характерные слова для каждой эмоции. Для примера посмотрим на  *Радость* и *Неуверенность*, у которых порядковые номера 1 и 4.

In [ ]:
labels = {1:'Радость', 4:'Неуверенность'}
coefficient_matrix = logreg.coef_

for i in [1, 4]:

    print(f"\n{labels[i]}:")

    feature_names = vect.get_feature_names_out() # word features
    order = coefficient_matrix[i].argsort() # ascending order for coefficients
    class_coefficients = coefficient_matrix[i][order][::-1][:5] # sort and extract top-5 coefficients
    feature_names = feature_names[order][::-1][:5] # words with the highest coefficients

    for feature, coefficient in zip(feature_names, class_coefficients):
      print(feature, coefficient.round(2))

# Деревья решений

Деревья решений — это одни из первых моделей машинного обучения, которые были известны человеку. Изначально их строили без специальных алгоритмов, а просто вручную.

Когда требовалось принять решение по проблеме, для которой построено дерево, человек брал и проходился по этому дереву.

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L03/out/manual_construction_of_decision_trees.png" width="900"></center>

<center><em>Предсказание типа линз для человека</em></center>

За принципом работы дерева решений стоит понятная интуиция. В каждом узле есть какой-то вопрос. Например, нормальное ли у человека слезоотделение, есть ли у него астигматизм и так далее. И, отвечая на каждый из этих вопросов, мы перемещаемся по дереву до тех пор, пока не придем к нужному типу линз.

Несмотря на возраст подхода, эти модели могут быть неожиданно эффективны и их можно автоматически строить с помощью алгоритмов и делать это достаточно быстро даже на больших объемах данных.

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L03/out/neuro_tree.jpeg" width="600"></center>

<center><em>"As long as Kaggle has been around, Anthony says, it has <font color=green >almost always</font> been <b>ensembles of decision trees that have won competitions</b>".</em></center>


В очень многих задачах, где нет какой-то локальной связанности, которая есть в изображениях, текстах и т.д., деревья решений эффективнее, чем нейронные сети.

## Принцип работы дерева решений

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L03/out/decision_tree_principle.png" width="900"></center>

Допустим, есть 2 признака, в данном случае вещественных. Для каждой точки мы создаем вопрос: признак $x_1$ больше $0.7$ или меньше? Если больше $0.7$, то это красная точка. Если меньше $0.7$, то идем во второй внутренний узел T2 и спрашиваем: признак $x_2$ меньше $0.5$ или больше? Если меньше $0.5$, то точка будет красная, в другом случае — синяя.

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L03/out/decision_trees_partitioning_space.png" width="700"></center>

<center><em>Разбиение пространства</em></center>

Дерево решений делит пространство признаков с помощью плоскостей на области, и в каждой из этих областей предсказывается константная величина.

## Классификация

Деревья решений, как правило, используются при решении 2-х типов задач — классификации и регрессии.
Первая — классификация. Предсказание может быть:


*   Жёстким (метка класса)
*   Мягким (вероятность класса)



<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L03/out/probability_estimation_by_decision_tree.png" width="700"></center>

В листе находятся объекты разных классов. Для того, чтобы оценить вероятность принадлежности объекта к какому-то определенному классу, можно просто число представителей данного класса поделить на общее число объектов. Это дает нам одну важную интуицию: желательно, чтобы в листе  было не очень мало объектов. Чем больше объектов, тем меньше ошибка.

Фактически, это оценка генеральной совокупности по ограниченной выборке. Можно использовать [распределение Бернулли 📚[wiki]](https://ru.wikipedia.org/wiki/Распределение_Бернулли) для двух классов или [мультиномиальное распределение 📚[wiki]](https://ru.wikipedia.org/wiki/Мультиномиальное_распределение) для большего количества классов.

### Для бинарных признаков

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L03/out/decision_tree_for_two_features.png" width="900"></center>

Представим, что мы решаем задачу бинарной классификации (есть инфаркт у человека или нет).

Возьмем в качестве первого вопроса признак "боль в груди". В 1-ом и во 2-ом листах получается разное распределение людей. В левом листе инфаркт более вероятен, в правом — менее вероятен.

Другим признаком может быть “как хорошо циркулирует кровь”, в таком случае тоже получится неплохое разделение. Последний признак — “есть ли атеросклероз”.

Мы получили три разбиения по разным признакам. Теперь мы бы хотели выбрать лучший признак, для этого нам нужно их сравнить.

Логично выбрать такое разбиение, которое дает нам "хорошие" узлы — те, в которых преимущественно сосредоточены объекты одного класса

Одна из используемых метрик называется [Gini ✏️[blog]](https://www.learndatasci.com/glossary/gini-impurity/) . Она считается по следующей формуле:

$$\large \text{Gini} = 1 - \sum_ip_i^2$$

Gini — мера [неопределенности ✏️[blog]](https://quantdare.com/decision-trees-gini-vs-entropy/) значения класса внутри узла. Соответственно, чем она ниже, тем лучше получившийся узел.

Посчитаем Gini для признака "Боль в груди":

$\text{Gini}_1 = 1- (\dfrac{105}{105+33})^2 - (\dfrac{33}{105+33})^2 = 0.364$

$\text{Gini}_2 = 1- (\dfrac{34}{34+125})^2 - (\dfrac{125}{34+125})^2 = 0.336$

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L03/out/compute_gini_for_binary_features.png" width="900"></center>

Такую метрику можно посчитать для каждого узла. Дальше посчитать метрику для следующего узла. Дальше можем оценить, насколько стал лучше результат в зависимости от используемого признака.

$$\large \text{Impurity decrease} = \text{Gini}_0 - (\frac{n_1}{n_1+n_2}\text{Gini}_1 + \frac{n_2}{n_1+n_2}\text{Gini}_2),$$

где $n_1, n_2$ — число объектов в листьях,

$ \quad\  \text{Gini}_0$ — чистота исходного узла.

**Боль в груди:**

 $\text{Impurity decrease} = 0.498 - (\dfrac{138}{138+159})\cdot 0.364 - (\dfrac{159}{138+159})\cdot 0.336 = 0.149$

**Хорошо циркулирует кровь:**

 $\text{Impurity decrease} = 0.498 - (\dfrac{164}{164+133})\cdot 0.349 - (\dfrac{133}{164+133})\cdot 0.373 = 0.138$

**Есть атеросклероз:**  

$\text{Impurity decrease} = 0.498 - (\dfrac{123}{123+174})\cdot 0.377 - (\dfrac{174}{123+174})\cdot 0.383 = 0.117$

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L03/out/compute_impurity_decrease.png" width="600"></center>

**Выбираем это разбиение как приводящее к наибольшему уменьшению Impurity**

Наибольший $\text{Impurity decrease}$ в признаке “боль в груди”. Значит, мы возьмем “боль в груди” как признак, на основании которого продолжим строить дерево.  

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L03/out/compute_gini_for_another_features.png" width="600"></center>

Каждый раз мы будем выбирать новые признаки. В одном листе один признак лучше разделяет его на 2, в другом — другой. Тут мы остановились при достижении глубины дерева 2.

Мы могли выбрать другой критерий остановки. Например, когда у нас получатся листья, в которых есть объекты только 1 класса. В этом случае не имеет смысла использовать другие разбиения. Либо может сложиться ситуация, когда разбиение по признаку, которое мы дополнительно взяли, ситуацию никак не улучшает.

### Для вещественных признаков

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L03/out/decision_tree_for_real_numbers.png" width="1000"></center>

Вещественный признак сортируется, после чего мы выбираем оптимальный порог для разбиения. Для этого считаем `impurity_decrease` для каждого порога и выбираем лучший.

## Регрессия


Для решения задач регрессии дерево строится практически так же, но есть несколько нюансов.

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L03/out/decision_tree_regression.png" width="700"></center>

Как лучше всего одним числом охарактеризовать все эти объекты, попавшие в один лист?

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L03/out/regression_metrics.png" width="900"></center>

<center><em>Способы формирования предсказания в листе для регрессии</em></center>

Как мы помним из статистики, у нас нет однозначного ответа на этот вопрос. Мы можем для распределения всех этих объектов в листе предсказывать наиболее частое значение, медиану, среднее значение.

Обычно предсказывают среднее значение, потому что с ним легче всего работать и его чаще всего используют в статистике.

Как корректно оценит **среднее** значение объектов **генеральной совокупности** для узла **по объектам тренировочной выборки**, попадающих в данный узел?

$$\large \overline{y} = \dfrac {\sum_i y_i} {n}$$

Нужно оценить дисперсию, которая показывает, насколько сильно мы можем ошибаться по сравнению с реальным (несмещенным) средним.

$$\large D(Y) = \dfrac {\sum_{i=1}^n(y_i-\overline{y})} {n-1} $$

Из-за неточности в оценке среднего при оценке дисперсии в знаменателе стоит $n-1$. Желательно иметь в каждом листе достаточное число объектов, чтобы компенсировать эту $-1$.

Теперь надо сформулировать критерий качества узла для регрессионного дерева. Мы хотим, чтобы значения в узле отличались как можно меньше, чтобы минимизировать ошибку предсказания (вычисленное нами **среднее является предсказанием** для объекта, попавшего в узел).

$\text{MSE}$ на тренировочной выборке, если мы предсказываем ее среднее, будет таким:

$$\large \text{MSE} = \frac 1 N \sum_{l=1}^L \sum_{i=1}^{n_l} (y_{li} - \overline{y_l})^2 = \frac 1 N \sum_{l=1}^L \sum_{i=1}^{n_l}\dfrac{n_l-1}{n_l}D(Y_l),$$

где $N$ — общее количество объектов в выборке, $L$ — общее число листьев, $n_l$ — число объектов в листе $l$.

Уменьшение дисперсии листьев $D(Y_l)$ приводит к уменьшению $\text{MSE}$.

Дополнительно будем взвешивать дисперсии на размер узла. Иначе самыми выгодными будут разбиения, отправляющие в один из узлов только один объект.

## Свойства деревьев решений

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L03/out/why_decision_tree_powerful_method.png" width="900"></center>

$\displaystyle \qquad \qquad \qquad \qquad h(x) = \sum \sigma(\dots \sum(w^Tx)) \qquad \qquad \qquad \qquad \qquad \qquad \qquad h(x) = \sum_dc_dI\{x \in R_d\}$

У деревьев решений есть еще одно хорошее свойство. Позже вы познакомитесь с замечательной [теоремой об универсальном аппроксиматоре 📚[wiki]](https://ru.wikipedia.org/wiki/%D0%A2%D0%B5%D0%BE%D1%80%D0%B5%D0%BC%D0%B0_%D0%A6%D1%8B%D0%B1%D0%B5%D0%BD%D0%BA%D0%BE). Ее суть в том, что нейросеть с одним скрытым слоем сможет аппроксимировать любую заданную гладкую функцию.

Для деревьев решений есть аналогичная теорема, говорящая о том, что дерево может аппроксимировать любую заданную кусочно-постоянную функцию.

Дерево решений, в отличие от нейронной сети, может адаптироваться к выборке любого размера, любому количеству признаков. Для нейронной сети эта теорема предполагает, что можно бесконечно увеличивать скрытые слои.

### Неустойчивость и переобучение деревьев<a class="anchor" style="autocontent" id="Неустойчивость-деревьев-решений"/><br>

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L03/out/instability_of_decision_trees.png" width="900"/></center>

Деревья решений не используются в чистом виде, потому что они неустойчивы. Если у нас есть данные, и мы удалим из них несколько объектов, то дерево решений может сильно поменяться.

Будем учиться отделять Злость от остальных эмоций.

In [ ]:
target = y != 1  # 0 for setosa, 1 - versicolor, 2 - virginica
# first set of points


Сделаем два разных разбиения на обучение и тест. И посмотрим, будут ли отличаться деревья, построенные для данных разбиений.

In [ ]:
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

x_train1, x_test1, y_train1, y_test1 = train_test_split(
    X, target, random_state=0
)
vect = TfidfVectorizer(min_df=0.0002, max_df=0.2)
X_train_vect1 = vect.fit_transform(x_train1)

clf1 = DecisionTreeClassifier(max_depth=2)
clf1.fit(X_train_vect1.toarray(), y_train1)

# second set of points
x_train2, x_test2, y_train2, y_test2 = train_test_split(
    X, target, random_state=42
)
vect = TfidfVectorizer(min_df=0.0002, max_df=0.2)
X_train_vect2 = vect.fit_transform(x_train2)

clf2 = DecisionTreeClassifier(max_depth=2)
clf2.fit(X_train_vect2.toarray(), y_train2)

cn = ['anger', 'joy', 'neutrality', 'sadness', 'uncertainty']
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5), dpi=100)
tree.plot_tree(clf1, class_names=cn, filled=True, ax=axes[0])
tree.plot_tree(clf2, class_names=cn, filled=True, ax=axes[1])
plt.show()

Видим, что даже деревья максимальной глубины 2 уже не совпадают между собой.

Если алгоритм при небольшом изменении признаков сильно меняет свое решение,  то это указывает на возможное переобучение. Алгоритм сильно реагирует на любой шум в данных— доверять его решениям опасно.

## Bias, Variance, Irreducible error

Можно показать, что ошибка любой модели раскладывается в сумму трех компонент:

$$ \large \text{Model error} = \text{Bias} + \text{Variance} + \text{Irreducible error} $$

### Bias

Обычно высокий bias имеют модели, которые недостаточно сложны по сравнению с реальной закономерностью данных. Например, реальная зависимость, которую мы наблюдаем, нелинейная, а мы пытаемся аппроксимировать ее прямой линией. В этом случае наше **решение** заведомо **смещено** (biased) в сторону линейной модели, и мы будем систематически ошибаться в сравнении с реальной моделью данных.

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L03/out/high_bias.png" width="600"/></center>

### Variance

Можно получить и обратную ситуацию. Если модель будет слишком сложная (в смысле своей выразительной способности) и "гибкая", то она сможет подстроиться под данные и выучить тренировочную выборку полностью. В этом случае модель будет подстраиваться под любой шум в данных и пытаться объяснить его какой-то сложной закономерностью.

Малое изменение в данных будет приводить к большим изменениям в прогнозе модели.

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L03/out/high_variance.png" width="600"/></center>

### Bias vs variance

В практических задачах не получается бесконечно уменьшать и Bias, и Variance — приходится искать **компромисс (bias-variance tradeoff)**. С какого-то момента при уменьшении Bias начнет увеличиваться Variance, и наоборот. Задача исследователя — найти точку оптимума.

Можно построить зависимость этих величин от сложности модели (capacity). По мере увеличения сложности Variance имеет тенденцию к возрастанию, а Bias — к убыванию. Более сложные модели подстраиваются под случайные шумы обучающей выборки, а более простые — не могут воспроизвести реальные закономерности.


<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L03/out/bias_variance_tradeoff.png" width="450"/></center>

Управлять эффектом variance и bias можно как с помощью выбора модели, так и с помощью выбора гиперпараметров модели.

Продемонстрируем источники компонент Bias и Variance на примере решения задачи регрессии зашумленной косинусоиды с помощью решающих деревьев. Создадим функцию для генерации небольшой обучающей выборки и отобразим ее на графике:

In [ ]:
np.random.seed(42)

num_points = 300
num_grid = 500
x_max = 3.14
plt.figure(figsize=(10, 6))


def get_sample(num_points, x_max, std=0.3, x_sample=None):
    if x_sample is None:
        x_sample = (np.random.rand(num_points) - 0.5) * 2 * x_max
    y_sample = np.cos(x_sample.flatten()) + np.random.randn(x_sample.shape[0]) * std
    return x_sample.reshape(-1, 1), y_sample


x_grid = np.linspace(-x_max, x_max, num_grid).reshape(-1, 1)
x_sample, y_sample = get_sample(num_points=num_points, x_max=x_max)
_, y_true = get_sample(num_points=num_points, x_max=x_max, std=0, x_sample=x_grid)

plt.scatter(x_sample, y_sample, c="#5D5DA6", alpha=0.5, label="Noised data")
plt.plot(x_grid, y_true, "#F9B041", linewidth=4, label="Real function")
plt.xlabel("X")
plt.ylabel("Y")
plt.legend()
plt.show()

Обучим три решающих дерева максимальной глубины (параметр `max_depth=None`) на разных выборках. Сравним предсказания моделей друг с другом и с реальной целевой функцией.

In [ ]:
from sklearn.tree import DecisionTreeRegressor

np.random.seed(42)

num_points = 30
num_models = 3
plt.figure(figsize=(24, 6))

model = DecisionTreeRegressor(max_depth=None)
y_pred = np.zeros((num_models, num_grid))
sample_color = ["#00E134", "#3A0DE2", "#FF00B3"]
for model_num in range(num_models):
    x_sample, y_sample = get_sample(num_points=num_points, x_max=x_max)
    model.fit(x_sample, y_sample)
    y_pred[model_num] = model.predict(x_grid)
    _, y_true = get_sample(num_points=num_points, x_max=x_max, std=0, x_sample=x_grid)

    plt.subplot(1, 3, model_num + 1)
    plt.scatter(
        x_sample, y_sample, c=sample_color[model_num], label=f"sample {model_num+1}"
    )
    plt.plot(
        x_grid,
        y_pred[model_num],
        c=sample_color[model_num],
        alpha=0.8,
        label=f"model trained on sample {model_num+1}",
    )
    plt.plot(x_grid, y_true, "#F9B041", linewidth=4, label="real mean")
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.ylim(-1.5, 1.8)
    plt.legend(loc="lower center")

Предсказания моделей отличаются друг от друга и от истинной кривой средних значений косинуса.

Обучим 1000 моделей c разной глубиной ($1, 5$ и максимальной) на разных подвыборках наших данных. Выберем одну тестовую точку $x=0$ и посмотрим, как предсказания моделей в этой точке распределены относительно истинного значения моделируемой функции.

In [ ]:
import matplotlib.gridspec as gridspec

num_models = 1000

for max_depth in [1, 5, None]:
    model = DecisionTreeRegressor(max_depth=max_depth)

    y_pred = np.zeros((num_models, num_grid))

    plt.figure(figsize=(10, 4))
    gs = gridspec.GridSpec(1, 2, width_ratios=[2, 1])
    plt.subplot(gs[0])

    for model_num in range(num_models):
        x_sample, y_sample = get_sample(num_points=num_points, x_max=x_max)
        model.fit(x_sample, y_sample)
        y_pred[model_num] = model.predict(x_grid)
        plt.plot(x_grid, y_pred[model_num], alpha=0.1, c="#2DA9E1", linewidth=1)

    _, y_true = get_sample(num_points=num_points, x_max=x_max, std=0, x_sample=x_grid)
    plt.plot(x_grid, y_true, c="#F9B041", linewidth=3, label="real mean")
    plt.axvline(x=x_grid[num_grid // 2], c="#5D5DA6", linewidth=3, label="X text point")
    plt.xlim((-x_max, x_max))
    plt.ylim((-1, 2))
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.gca().set_title(f"{num_models} models: max depth = {max_depth}")
    plt.legend(loc="upper right")

    plt.subplot(gs[1])
    var = y_pred[:, num_grid // 2].var()
    bias = np.abs(y_true[num_grid // 2] - y_pred[:, num_grid // 2].mean())
    plt.hist(
        y_pred[:, num_grid // 2],
        bins=15,
        color="#2DA9E1",
        alpha=0.5,
        orientation="horizontal",
        label=f"predictions: \nvar = {var:.2f}\nbias = {bias:.2f}",
    )
    plt.axhline(y=y_true[num_grid // 2], c="#F9B041", linewidth=3, label="real mean")
    plt.ylim((-1, 2))
    plt.xlabel("hist counts")
    plt.ylabel("Y")
    plt.gca().set_title(f"predictions at test point")
    plt.legend(loc="upper left")
    plt.tight_layout()
    plt.show()

По мере увеличения глубины дерева:
- уменьшается абсолютное значение сдвига среднего значения предсказаний моделей относительно истинного значения (Bias) — в среднем предсказания моделей становятся более точными,
- увеличивается дисперсия предсказаний моделей (Variance) — предсказания моделей становятся менее устойчивы и сильнее зависят от конкретной обучающей выборки.



*   **Деревья малой глубины имеют малую сложность и высокий Bias.**
*   **Деревья большой глубины имеют высокую сложность и высокий Variance.**



# Ансамбли

## Корректирующий код

Пусть у нас есть сигнал:


<font color=2BA8E0 size=30>1110110011</font>

Но при передаче на другое устройство в нем могут возникать ошибки:

<font color=2BA8E0 size=30>1</font><font color=red size=30>0</font><font color=2BA8E0 size=30>1011</font><font color=red size=30>1</font><font color=2BA8E0 size=30>011</font>

Самое простое решение возникшей проблемы:

 1. Шум, который вносит ошибки, скорее всего не зависит от места в сигнале
 2. Передадим 3 раза один и тот же сигнал

<font color=2BA8E0 size=30>1</font><font color=red size=30>0</font><font color=2BA8E0 size=30>1011</font><font color=red size=30>1</font><font color=2BA8E0 size=30>011</font>

<font color=2BA8E0 size=30>1110</font><font color=red size=30>0</font><font color=2BA8E0 size=30>10011</font>

<font color=2BA8E0 size=30>11</font><font color=red size=30>0</font><font color=2BA8E0 size=30>0110</font><font color=red size=30>1</font> <font color=2BA8E0 size=30>11</font>



 3. Усредним, что получилось (в каждом случае возьмем наиболее часто встречающуюся цифру)

<font color=2BA8E0 size=30>1110110011</font>


 4. С большой долей вероятности итоговый сигнал восстановится
 5. Чем больше копий сигналов передастся, тем выше вероятность, что сигнал восстановится полностью корректно

## Усреднение предсказания классификаторов

Постановка задачи:

Есть 10 объектов, в реальности все принадлежат классу 1:

<font color=2BA8E0 size=30>1111111111</font>

Пусть у нас есть три **независимых** классификатора A, B и C. Каждый предсказывает 1 в 70% случаев.

Мы хотим получить общий классификатор на основании этих трех.

Мы хотим получить предсказание базовых классификаторов и применить к ним какую-то функцию, которая выдаст итоговый ответ. Вид этой функции задается заранее.

Будем просто усреднять предсказание наших классификаторов:

$$\large h(x) = \dfrac 1 T \sum_{i=1}^{T}a_i(x) $$

Простое голосование

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L03/out/simple_voting.png" width="450"/></center>

Посчитаем вероятность того, что:

 1. Все три классификатора верны: $0.7 * 0.7 * 0.7 = 0.3429$
 2. Два классификатора верны: $0.7 * 0.7 * 0.3 + 0.7 * 0.3 * 0.7 + 0.3 * 0.7 * 0.7 = 0.4409$

Таким образом, если брать большинство голосов, то мы будем в 78% случаев предсказывать верно. Мы взяли 3 классификатора, которые сами по себе были не очень хорошими, и получили классификатор лучшего качества.
Если взять больше классификаторов, то ситуация будет еще лучше.


Пусть теперь у нас три классификатора, выдающие следующие предсказания

<font color=2BA8E0 size=30>11111111</font><font color=red size=30>00</font> — $80\%$ точность

<font color=2BA8E0 size=30>11111111</font><font color=red size=30>00</font> — $80\%$ точность

<font color=2BA8E0 size=30>1</font></font><font color=red size=30>0</font><font color=2BA8E0 size=30>111111</font><font color=red size=30>00</font> — $70\%$ точность

Если объединим предсказания, то получим:

<font color=2BA8E0 size=30>11111111</font><font color=red size=30>00</font> — $80\%$ точность

Потому что очень **высокая зависимость предсказаний**. Выше видно, что два классификатора предсказывают абсолютно одинаково. Вероятность, что они делают это случайно, очень мала.

А вот если возьмем такие классификаторы, то все получится:

<font color=2BA8E0 size=30>11111111</font><font color=red size=30>00</font> — $80\%$ точность

<font color=red size=30>0</font><font color=2BA8E0 size=30>111</font><font color=red size=30>0</font><font color=2BA8E0 size=30>111</font><font color=red size=30>0</font><font color=2BA8E0 size=30>1</font> — $70\%$ точность

<font color=2BA8E0 size=30>1</font><font color=red size=30>000</font><font color=2BA8E0 size=30>1</font><font color=red size=30>0</font><font color=2BA8E0 size=30>1111</font> — $60\%$ точность


Усреднение:

<font color=2BA8E0 size=30>11111111</font><font color=red size=30>0</font><font color=2BA8E0 size=30>1</font> — $90\%$ точность

### Зависимость качества ансамбля от качества индивидуального предсказателя и от числа предсказателей

In [ ]:
def get_predictions(y_real, p, cnt):
    size = y_real.shape[0]
    guessed = np.random.choice([True, False], (cnt, size), p=[p, 1 - p])
    y = np.repeat(y_real.reshape(1, -1), cnt, axis=0)
    y[~guessed] = 1 - y[~guessed]
    return y

In [ ]:
import pandas as pd
import seaborn as sns

size = 1000
reps = 10

cnt_base_predictors = [1] + list(range(5, 105, 5))
single_qual = [0.45, 0.5, 0.51, 0.55, 0.6, 0.75, 0.9]

dt = {"cnt": [], "single_qual": [], "accuracy": []}

for i in range(reps):
    y_real = np.random.choice([0, 1], size)
    for cnt in cnt_base_predictors:
        for p in single_qual:
            preds = get_predictions(y_real, p, cnt)
            voting = np.round(preds.mean(axis=0))
            accuracy = (y_real == voting).mean()
            dt["cnt"].append(cnt)
            dt["single_qual"].append(f"{p:.02}")
            dt["accuracy"].append(accuracy)

results = pd.DataFrame(dt)

plt.figure(figsize=(16, 6))

sns.lineplot(data=results, x="cnt", y="accuracy", hue="single_qual", lw=3, alpha=0.5)
plt.xlabel("Number of base classifiers", size=20)
plt.ylabel("Accuracy", size=20)
plt.legend(loc="best", fontsize=12, title="Single classifier quality")
plt.show()

Видим, что:
1. Чем лучше базовый классификатор, тем меньше нужно классификаторов при прочих равных условиях для достижения высокого качества.
2. Если качество базового классификатора даже чуть больше 0.5, то качество ансамбля растет с увеличением числа моделей в ансамбле.
3. Если качество базового классификатора неотличимо от случайного (0.5), то качество ансамбля будет оставаться равным 0.5.
4. Если качество базового классификатора ниже случайного (0.5), то качество ансамбля стремится к 0.

### Коррелированность моделей


Посмотрим, как зависит качество предсказания от коррелированности предсказателей. Конкретно — от ожидаемой коррелированности вероятностей ошибиться на данном объекте для любой взятой пары классификаторов из ансамбля.

In [ ]:
import scipy


def get_correlated_predictions(y_real, p, cnt, r):
    size = y_real.shape[0]
    x1 = np.random.uniform(0, 1, size)
    x2 = np.random.uniform(0, 1, (cnt, size))
    q = np.sqrt(r)
    y = q * x1 + (1 - q**2) ** 0.5 * x2  # y variables now correlated with correlation=r
    y_mod = np.zeros_like(y)
    for i in range(y.shape[0]):
        y_mod[i] = scipy.stats.rankdata(y[i])

    y = y_mod / size  # back to uniform, slightly affects correlations

    y_pred = np.repeat(y_real.reshape(1, -1), cnt, axis=0)
    y_pred[y < 1 - p] = 1 - y_pred[y < 1 - p]  # to predictions, affects correlations
    return y_pred

In [ ]:
np.random.seed(42)
x = np.arange(0, 1, 0.05)
accuracy = np.zeros_like(x)
p = 0.7
cnt = 100
for ind, r in enumerate(x):
    preds = get_correlated_predictions(y_real, p, cnt, r)
    voting = np.round(preds.mean(axis=0))
    accuracy[ind] = (y_real == voting).mean()

plt.figure(figsize=(16, 6))
plt.title(f"Accuracy of {cnt} classifiers ensemble", size=20)
plt.xlabel("Correlation among classifiers", size=20)
plt.ylabel("Accuracy", size=20)
plt.axhline(y=p, color="red", lw=5, ls="--", label="Single classifier")
sns.lineplot(x=x, y=accuracy, lw=5, label="Ensemble")
plt.legend(fontsize=20)
plt.show()

Видим, что по мере увеличения коррелированности моделей качество все больше и больше приближается к качеству одной базовой модели.

## Случайный лес

Так мы приходим к идее случайного леса. Мы будем брат множество "плохих" деревьев, каждое из которых переобучается, каждое из которых не похоже на соседнее и объединять их ответы. Нам достаточно того, чтобы отдельные деревья выдавали ответ лучше случайного.

Чтобы добиться ещё лучшего качества леса, будем при построении деревьев в узлах рассматривать случайное подмножество признаков.

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L03/out/random_forest.png" width="900"/></center>

Случайный лес устойчив к шуму и может обрабатывать данные с пропусками.

Для того, чтобы лес работал лучше, следует придерживаться следующее правил:


*   Больше деревьев в лесу
*   Чем глубже, тем лучше
*   Не ограничивать минимальный размер листа

Случайный лес переобучить сложно, но можно.

In [ ]:
import pandas as pd
emo = pd.read_csv('Emotion_Classification_Russian.csv')

emo['num_label'] = emo['label'].astype('category').cat.codes
X, y = emo['lemmatized'], emo['num_label']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify = y)

vect = TfidfVectorizer(min_df=0.0002, max_df=0.2)
X_train_vect = vect.fit_transform(X_train)
X_test_vect = vect.transform(X_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

models_rf = {}

# this can be done faster, see warm_start parameter for this
# (https://stackoverflow.com/questions/42757892/how-to-use-warm-start)
for n_estimators in [3, 5, 10, 50, 100]:
    models_rf[f"RF{n_estimators}"] = RandomForestClassifier(
        n_estimators=n_estimators, random_state=42, n_jobs=-1
    )  # run in parallel

Создадим несколько случайных лесов с различным количеством деревьев и простое дерево решений для сравнения с ним (baseline).

Обучим все модели.
Для этого напишем вспомогательную функцию, которая будет обучать переданные ей модели и считать для них качество на тесте.

In [ ]:
from sklearn.metrics import f1_score

def train_and_test_classifier(models, x_train, y_train, x_test, y_test, verb=True):
    scores = {}
    for name, model in models.items():
        model.fit(x_train.toarray(), y_train)  # train the model
        y_pred = model.predict(x_test.toarray())  # get predictions
        scores[name] = f1_score(y_test, y_pred, average = 'weighted')
        print(f"Fitted {name} with F1 score {scores[name]:.3f}")
    print (scores)
    results = pd.DataFrame(scores, index=[0])

    return results


results_rf = train_and_test_classifier(models_rf, X_train_vect, y_train, X_test_vect, y_test)

In [ ]:
plt.figure(figsize=(12, 4))
sns.barplot(data=results_rf)
plt.ylabel("F1", size=20)
plt.xlabel("n_estimators", size=20)
plt.title("Number of estimators vs F1", size=20)
plt.tick_params(axis="both", which="major", labelsize=20)
plt.show()

In [ ]:
from sklearn.metrics import classification_report
target_names = ['anger', 'joy', 'neutrality', 'sadness', 'uncertainty']
rf_pred = models_rf["RF100"].predict(X_test_vect.toarray())
print(classification_report(y_test, rf_pred, target_names=target_names))

## Boosting

Бустинг — это еще один способ ансамблирования моделей. В отличие от случайного леса, где каждая модель в ансамбле строится независимо, в бустинге построение очередной модели зависит от уже состоящих в ансамбле моделей. Каждая следующая модель в ансамбле стремится улучшить предсказание всего ансамбля.

Бустинг позволяет нам на основе большого числа "слабых" моделей получить одну "сильную". Под слабыми моделями подразумеваем модели, точность которых может быть лишь немногим выше случайного угадывания.

В качестве моделей традиционно используются деревья решений, но не большой глубины, а наоборот — маленькой, чтобы каждое из них не могло выучить выборку хорошо.








<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L03/out/join_weak_learners.png" width="650"/></center>

Из всех бустингов больше всего прославился и доказал свою эффективность **градиентный бустинг**. Он позволяет получить решение, которое сложно побить другими видами моделей.



## Gradient boosting (градиентный бустинг)

Начнем с интуиции. Предположим, что мы играем в гольф. Наша задача — загнать мячик в лунку, причем, когда мячик далеко от нее, мы можем ударить посильнее, чтобы сократить расстояние, но когда мы уже близко к лунке, то стараемся бить клюшкой аккуратнее, чтобы не промахнуться. После каждого удара мы оцениваем расстояние до лунки и приближаем мячик в ее сторону. Применительно к нашей теме, удар клюшкой по мячику — это каждая модель в градиентном бустинге.

[[colab] 🥨 Демонстрация интуиции градиентного бустинга](https://colab.research.google.com/drive/1hILdJzsuAsXabA4aIwtb9RGgUn_bvhCL)

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L03/out/summarize_predictions_of_ensemble_models.png" width="650"/></center>

Цель $i$-той модели в ансамбле — скорректировать ошибки предыдущих ${i-1}$ моделей. В результате, когда мы суммируем вклады всех моделей ансамбля, получается хорошее предсказание.

Давайте формализуем, что нам нужно, чтобы обучить алгоритм градиентного бустинга:

- набор данных $\left\{\left(x_i, y_i\right)\right\}_{i=1, \ldots, n}$,
- число итераций $M$ (оно же количество моделей),
- выбор дифференцируемой функции потерь $L(y, f)$,
- выбор семейства функций базовых алгоритмов $h(x, \theta)$ с процедурой их обучения.

Минимизировать ошибку будем с помощью градиентного спуска.

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L03/out/gradient_boosting.png" width="600"/></center>

Разберем пошагово.

Начинаем с первого предсказания. Можем выбрать на самом деле любое число, например, среднее значение:

$$ \large \hat{f}(x)=\hat{f}_0, \hat{f}_0=\gamma, \gamma \in \mathbb{R}
 \tag{1}$$

Либо подобрать с наименьшей ошибкой:

$$\large \hat{f}_0=\underset{\gamma}{\arg \min } \sum_{i=1}^n L\left(y_i, \gamma\right) $$

Посмотрим на графике, как бы могла выглядеть наша зависимость. Посчитаем ошибку и отобразим наше предсказание.



<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L03/out/gb_explanation_step1.png" width="700"/></center>

В нашем примере мы видим, что нужно двигаться вправо, чтобы уменьшить ошибку. В действительности мы этого не знаем, но можем посчитать градиент и узнать направление возрастания функции ошибки. Нас же интересует обратное направление или градиент со знаком минус (антиградиент).

$$\large r_{i t}=-\frac{\partial L\left(y_i, f\left(x_i\right)\right)}{\partial f\left(x_i\right)} \quad \tag{2}$$

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L03/out/gb_explanation_step2.png" width="700"/></center>

Строим новый базовый алгоритм $h_t(x)$, который обучается на то, чтобы уменьшить ошибку от текущего состояния ансамбля, и в качестве целевой переменной для него берем антиградиент функции потерь  $\left\{\left(x_i, r_{i t}\right)\right\}_{i=1, \ldots, n}$

$$\large h_t(x) = \underset{\theta}{\arg \min } \sum_{i=1}^n L\left(h(x_i, \theta), r_{it}\right) \tag{3}$$

Подбираем оптимальный параметр $\large \rho$. Этот параметр будет разным у каждого дерева в нашем ансамбле, в отличие от learning rate:

 $$\large \rho_t=\underset{\rho}{\arg \min } \sum_{i=1}^n L\left(y_i, \hat{f}\left(x_i\right)+\rho \cdot h_t\left(x_i, \theta\right)\right) \tag{4}$$

Сдвигаем предсказание в сторону уменьшения ошибки, где $\lambda$ — это learning rate:

 $$\large \hat{f}_t(x)= \lambda \cdot \rho_t \cdot h_t(x) \tag{5}$$

Обновляем текущее приближение $\hat{f}(x)$:
$$ \large
\hat{f}(x) \leftarrow \hat{f}(x)+\hat{f}_t(x)=\sum_{i=0}^t \hat{f}_i(x) \tag{6}
$$

Далее повторяем шаги 2–6, пока не получим требуемое качество, и собираем итоговый ансамбль $\hat{f}(x)$:

$$
\large
\hat{f}(x)=\sum_{i=0}^M \hat{f}_i(x)
$$

[[blog] ✏️ Подробнее о градиентном бустинге](https://habr.com/ru/company/ods/blog/327250/)



[[demo] 🎮 Gradient Boosting explained](https://arogozhnikov.github.io/2016/06/24/gradient_boosting_explained.html)

[[demo] 🎮 Gradient Boosting Interactive Playground](https://arogozhnikov.github.io/2016/07/05/gradient_boosting_playground.html)

Когда мы говорили про базовые модели, то предполагали, что этими базовыми моделями являются деревья решений. Однако это необязательно, можно использовать и другие алгоритмы в качестве базовых моделей.

Деревья решений имеют ряд преимуществ:

- Гибкость — деревья решений могут описывать сложные нелинейные взаимосвязи между объектами и целевой переменной. Они могут обрабатывать как числовые, так и категориальные данные и работать с пропущенными значениями.

- Интерпретируемость — деревья решений просты в понимании и интерпретации. Они дают четкое представление о том, как модель пришла к определенному прогнозу.

- Устойчивость к выбросам — деревья решений менее чувствительны к выбросам, чем другие алгоритмы машинного обучения. Они способны изолировать их и предотвратить их влияние на всю модель.

- Эффективность — деревья решений могут быть построены быстро, они способны обрабатывать большие наборы данных.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
models = {}

models["DT"] = DecisionTreeClassifier(
    max_depth=10,
    min_samples_leaf=10,
)

models["RF"] = RandomForestClassifier(
    n_estimators=100,  # for better result set to 1000
    max_depth=None,
    min_samples_leaf=1,
    n_jobs=-1,
    random_state=42,
)

# models["GradientBoosting"] = GradientBoostingClassifier(
#     learning_rate=0.1,  # for better result set to 0.05
#     n_estimators=100,  # for better result set to 1000
#     random_state=42,
# )

models["GradientBoosting"] = GradientBoostingClassifier(
    learning_rate=0.5,  # for better result set to 0.05
    n_estimators=10,  # for better result set to 1000
    random_state=42,
)

results_gb = train_and_test_classifier(models, X_train_vect, y_train, X_test_vect, y_test)

In [ ]:
from sklearn.metrics import classification_report
target_names = ['anger', 'joy', 'neutrality', 'sadness', 'uncertainty']
gb_pred = models["GradientBoosting"].predict(X_test_vect.toarray())
print(classification_report(y_test, gb_pred, target_names=target_names))

In [ ]:
plt.figure(figsize=(12, 4))
ax = sns.barplot(data=results_gb)
plt.xlabel("", size=20)
plt.ylabel("F1", size=20)
plt.title("DT vs RF vs GB", size=25)
plt.xticks(size=20)
plt.show()

### Переобучение

В то же время **Gradient boosting**, в отличие от случайного леса, может сильно переобучиться. Это важно понимать. Для небольших датасетов часто может оказаться, что случайный лес дает более надежные результаты.

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L03/out/gradient_boosting_overfitting.png" width="600"/></center>

В отличии от случайного леса, для градиентного бустинга п**одходят слабые модели** (деревья с малой глубиной).

### Shrinkage (learning rate)

Как у градиентного спуска есть learning rate, который определяет силу каждого нашего следующего шага,  так и у градиентного бустинга есть параметр, который называется shrinkage, на который мы домножаем вес, с которым добавляются новые модели в ансамбль.

Если не домножать вес каждой модели дополнительно на этот параметр, то мы можем попасть в ситуацию, когда будем пролетать мимо минимума функции ошибки (та же опасность, что и в обычном градиентном спуске).

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L03/out/shrinkage_learning_rate_for_gradient_boosting.png" width="450"/></center>

Как и в случае с градиентным спуском, **learning rate** влияет не только на то, как быстро мы станем переобучаться, но и на глубину минимума, который мы найдем.

## Модификации градиентного бустинга

Есть много модификаций градиентного бустинга. В отличие от реализации в `sklearn`, большая часть из них умеет параллелиться на CPU или даже на GPU.

Поэтому при работе с реальными данными использовать градиентный бустинг из `sklearn` не стоит. XGBoost и/или LigthGBM дадут результат как правило лучше и быстрее.

### CatBoost

[[doc] 🛠️ Официальная документация](https://catboost.ai/en/docs/)

Разработан Яндексом.

1. Хорошо умеет работать с категориальными признаками. Если у вас много категориальных признаков, он может дать существенный выигрыш.
2. Умеет работать с текстом без предварительной обработки. Достаточно указать, что колонка содержит текстовый признак.
3. По умолчанию использует в качестве модели модификацию обычного дерева решения — Symmetric Tree, которое менее склонно к переобучению.

In [ ]:
!pip install -q catboost

In [ ]:
!wget -q https://edunet.kea.su/repo/EduNet_NLP-web_dependencies/datasets/Emotion_Classification_Russian.csv

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

emo = pd.read_csv('Emotion_Classification_Russian.csv')

emo['num_label'] = emo['label'].astype('category').cat.codes
X, y = emo['lemmatized'], emo['num_label']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify = y)

vect = TfidfVectorizer(min_df=0.0002, max_df=0.2)
X_train_vect = vect.fit_transform(X_train)
X_test_vect = vect.transform(X_test)

In [ ]:
from catboost import CatBoostClassifier, Pool

models_cb = {}
models_cb["CatBoost"] = CatBoostClassifier(
    iterations=200, #2000
    learning_rate=0.1,
    random_state=42,
    verbose=0,
    task_type="GPU",
)
# task_type="GPU") # can use gpu, but no parallel-cpu option

results_cb = train_and_test_classifier(models_cb, X_train_vect, y_train, X_test_vect, y_test)

In [ ]:
from sklearn.metrics import classification_report
target_names = ['anger', 'joy', 'neutrality', 'sadness', 'uncertainty']
cb_pred = models_cb["CatBoost"].predict(X_test_vect.toarray())
print(classification_report(y_test, cb_pred, target_names=target_names))

In [ ]:
target_col = 'num_label'
text_cols = ['lemmatized']

In [ ]:
X_train.shape

In [ ]:
emo = pd.read_csv('Emotion_Classification_Russian.csv')

emo['num_label'] = emo['label'].astype('category').cat.codes
X, y = emo['lemmatized'], emo['num_label']

In [ ]:
emo

In [ ]:
import numpy as np
msk = np.random.rand(len(emo)) < 0.8

train = emo[msk]
test = emo[~msk]

train = train.reset_index(drop=True)
test = test.reset_index(drop=True)


In [ ]:
X_train,
X_test,
y_train,
y_test = train.drop (columns = ['num_label', 'label', 'text']), test.drop(columns = ['num_label', 'label', 'text']), train['num_label'], test['num_label']

In [ ]:
from sklearn.model_selection import StratifiedKFold

y_preds = []
models = []
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

catboost_params = {
    'iterations': 100,#1000
    'learning_rate': 0.1,
    'objective': 'MultiClass',
    #'eval_metric': 'F1', #'Logloss'
    'custom_metric': 'F1',
    'task_type': 'GPU',
    'early_stopping_rounds': 10,
    'use_best_model': True,
    'verbose': 100
}

for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train, y_train)):
    X_tr = X_train.loc[train_index]
    X_val = X_train.loc[valid_index]
    y_tr = y_train[train_index]
    y_val = y_train[valid_index]

    train_pool = Pool(
        X_tr,
        y_tr,
        #cat_features=categorical_cols,
        text_features=text_cols,
        feature_names=list(X_tr)
    )
    valid_pool = Pool(
        X_val,
        y_val,
        #cat_features=categorical_cols,
        text_features=text_cols,
        feature_names=list(X_tr)
    )

    model = CatBoostClassifier(**catboost_params)
    model.fit(train_pool, eval_set=valid_pool)

    y_pred = model.predict(X_test)
    y_preds.append(y_pred)
    models.append(model)

In [ ]:
y_test.shape

In [ ]:
y_preds[0]

In [ ]:
np.array(y_preds).shape

In [ ]:
#cb_pred = models_cb["CatBoost"].predict(X_test_vect.toarray())
print(classification_report(y_test, y_preds[-1], target_names=target_names))

### Optuna

Для оптимизации гиперпараметров существуют готовые решения, которые используют различные методы black-box оптимизации. Разберем одну из наиболее популярных библиотек — [**Optuna** 🛠️[doc]](https://optuna.org/).

Есть интеграции с scikit-learn, XGBoost и LightGBM, но мы рассмотрим общий случай, который подойдет для любой модели, даже нейронной сети.

In [ ]:
!pip install --quiet optuna

In [ ]:
import optuna
from optuna.samplers import TPESampler
from sklearn.model_selection import cross_val_score, KFold

# Define function which will optimized


def objective(trial):
    # boundaries for the optimizer's
    depth = trial.suggest_int("depth", 5, 8, step=1)
    min_data_in_leaf = trial.suggest_int("min_data_in_leaf", 6, 10, step=2)
    l2_leaf_reg = trial.suggest_categorical("l2_leaf_reg", [2, 5, 7, 10])
    random_strength = trial.suggest_float("random_strength", 1, 2)

    # create new model(and all parameters) every iteration
    model = CatBoostClassifier(
        iterations=200,#
        learning_rate=0.1,
        depth=depth,
        min_data_in_leaf=min_data_in_leaf,
        l2_leaf_reg=l2_leaf_reg,
        random_strength=random_strength,
        random_state=42,
        verbose=0,
    )
    kf = KFold(n_splits=3, shuffle=True, random_state=42)
    f1 = cross_val_score(
        model, X_train_vect.toarray(), y_train, cv=kf, scoring="f1"
    ).mean()
    error = f1

    return error


# Create "exploration"
study = optuna.create_study(
    direction="minimize", study_name="Optimizer", sampler=TPESampler(42)
)

study.optimize(
    objective, n_trials=7
)  # The more iterations, the higher the chances of catching the most optimal hyperparameters

Давайте посмотрим на историю оптимизации нашей метрики:

In [ ]:
optuna.visualization.plot_optimization_history(study)

Можем посмотреть, с какими параметрами какие результаты получились, и сделать выводы, в каких диапазонах лучше подбирать параметры:

In [ ]:
params = ["depth", "min_data_in_leaf", "l2_leaf_reg", "random_strength"]
optuna.visualization.plot_slice(study, params=params, target_name="MSE")

Выведем лучшие параметры:

In [ ]:
# show best params
study.best_params

Optuna оценивает важность параметров для лучшего результата:

In [ ]:
optuna.visualization.plot_param_importances(study)

Обучим CatBoost на значениях, подобранных Optuna:

In [ ]:
models_add5 = {}
models_add5["CatBoost tuned"] = catboost.CatBoostRegressor(
    iterations=2000,
    learning_rate=0.1,
    depth=study.best_params["depth"],
    min_data_in_leaf=study.best_params["min_data_in_leaf"],
    l2_leaf_reg=study.best_params["l2_leaf_reg"],
    random_strength=study.best_params["random_strength"],
    random_state=42,
    verbose=0,
)
# task_type="GPU") # can use gpu, but no parallel-cpu option

cat_tuned_add = train_and_test_classifier(models_add5, x_train, y_train, x_test, y_test)

In [ ]:
plt.figure(figsize=(16, 6))
ax = sns.boxplot(data=pd.concat([xgb_add2, lgb_add, cat_add, cat_tuned_add]))
plt.xlabel("", size=20)
plt.ylabel("MSE", size=20)
plt.title("Catboost tuned and others", size=20)
plt.xticks(size=20)
plt.show()

Качество незначительно улучшилось.

#Проблемы при работе с реальными данными

В реальных задачах вы столкнетесь с широким спектром трудностей, и одним из первых этапов будет подготовка данных. Уже на этом этапе есть ряд возможных проблем, например:

- **Нехватка данных** — у вас может быть мало данных. Более того, понять, какой объем данных необходим — тоже отдельная задача. Другая вариация похожей проблемы — это большой объем данных, но без разметки.

- **Некачественная разметка** —  даже в широко известных MNIST, CIFAR-10 и ImageNet есть [ошибки в разметке 🎮[demo]](https://labelerrors.com/). В вашем датасете они тоже будут. Важно понимать, что разметка может не являться бесспорным эталоном.

- **Шум в данных** — полезно подумать о потенциальной зашумленности данных. Например, когда люди заполняют таблицы, они могут допускать ошибки. Хорошо понимать, какого рода ошибки могут содержаться в ваших данных.

- **Несбалансированность датасета** — какие-то классы могут быть плохо представлены (**минорные классы**). Например, если в вашем датасете будет всего 10 фотографий одного класса и 10 000 другого, то модели будет очень заманчиво вообще не пытаться определять минорный класс (всего 0.1% ошибок).

- **Ковариантный сдвиг** — явление, когда признаки тренировочной и тестовой выборок **распределены по-разному**. Из-за этого мы можем получать плохие предсказания на тестовой выборке, когда модель просто не знает, что признаки могут находиться в другом распределении.

<img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L04/out/covariate_shift.png" width="600">

*Практический совет: для быстрого обнаружения ковариантного сдвига можно **обучить модель**, которая будет предсказывать, относится ли объект к **train** или **test** выборке. Если модель легко делит данные по такому принципу, то имеет смысл визуализировать значения признаков, по которым она это делает.*

- **Малое число источников данных** — проблема, родственная предыдущей. В вашем датасете могут быть данные только от одного прибора или одной модели прибора. Могут быть данные, снятые только одним специалистом, или в одной больнице, или только у взрослых. Это также может влиять на способность вашего алгоритма обобщать полученное решение и требует пристального внимания.

- **Грязные данные** — в данных могут быть полные/неполные дубликаты, пропуски, ошибки форматов, перемешанные столбцы и многое другое. Такие проблемы могут быть как естественной характеристикой (нет данных о каких-то объектах, отсюда пропуски), так и ошибкой при агрегации данных из разных источников. Важно подумать о проверках и тестах, чтобы быть уверенным в качестве.

Все это приводит к целому спектру сложностей, из которых самой типичной будет переобучение модели — какую бы простую модель вы не взяли, она все равно будет выучивать искажения вашего датасета.

## Дисбаланс классов

Прежде всего надо убедиться, что датасет сбалансирован:

In [ ]:
import torch
import matplotlib.pyplot as plt
from sklearn.datasets import load_wine


def show_class_balance(y, classes):
    _, counts = torch.unique(torch.tensor(y), return_counts=True)
    plt.bar(classes, counts)
    plt.ylabel("n_samples")
    plt.ylim([0, 75])
    plt.show()


wine = load_wine()
classes = wine.target_names

show_class_balance(wine.target, classes)

Разница в 10–20% будет незначительна, поэтому для наглядности мы искусственно разбалансируем наш датасет при помощи метода `make_imbalance` [🛠️[doc]](https://imbalanced-learn.org/stable/references/generated/imblearn.datasets.make_imbalance.html) из библиотеки [imbalanced-learn 🛠️[doc]](https://imbalanced-learn.org/stable/).

In [ ]:
from imblearn.datasets import make_imbalance

x, y = make_imbalance(
    wine.data, wine.target, sampling_strategy={0: 10, 1: 70, 2: 40}, random_state=42
)
show_class_balance(y, classes)

### Изменение баланса класса сэмплированием

Если в данных нехватка именно конкретного класса, то можно бороться с этим при помощи разных способов сэмплирования.

Важно понимать, что в большинстве случаев данные, полученные таким способом, должны использоваться в качестве **обучающего набора**, но ни в коем случае не в качестве **валидации** или **теста**.


#### Дублирование примеров меньшего класса (oversampling)

Мы можем увеличить число объектов меньшего класса за счет дублирования.

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L04/out/oversampling_scheme.png" width="600"></center>

<center><em>Дублирование примеров меньшего класса</em></center>



Такой oversampling может быть выполнен с помощью класса `RandomOverSampler` [🛠️[doc]](https://imbalanced-learn.org/stable/references/generated/imblearn.over_sampling.RandomOverSampler.html) из imbalanced-learn, как показано ниже:

In [ ]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=0)
x_ros, y_ros = ros.fit_resample(x, y)

show_class_balance(y_ros, classes)

#### Уменьшение числа примеров большего класса (undersampling)

Аналогично, можно взять для обучения не всех представителей большего класса.

<center><img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L04/out/undersampling_scheme.png" width="600"></center>

<center><em>Удаление примеров преобладающего класса</em></center>

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(random_state=42)
x_res, y_res = rus.fit_resample(x, y)

show_class_balance(y_res, classes)

Минус подхода: мы можем выбросить важных представителей большего класса, ответственных за существенное улучшение генерализации, и из-за этого качество модели существенно ухудшится.

Выбрасывать объекты большего класса  можно разными способами. Например, кластеризовать объекты большего класса и брать по заданному количеству объектов из каждого кластера.

## Борьба с переобучением

__Сложность модели__ (*model complexity*) — важный гиперпараметр. В частности, для линейных моделей сложность может быть представлена **количеством параметров**, например, для полиномиальных моделей — степенью полинома.

Сложность модели связана с **ошибкой обобщения** (_generalization error_):
- **Cлишком простой модели** не будет хватать **количества параметров для обобщения сложной закономерности в данных**, что приведёт к большой ошибке обобщения.
- **Избыточная сложность модели** также приводит к большой ошибке обобщения за счет того, что в силу своей сложности модель начинает **пытаться искать закономерности в шуме**, добиваясь большей точности на тренировочных данных, теряя при этом часть обобщающей способности.



<img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L02/out/model_complexity.png" width="500">

Параметры модели задают некоторую **аппроксимацию целевой функции**. Аппроксимировать целевую функцию можно несколькими способами, например:
1. Использовать все имеющиеся данные и провести ее строго **через все точки**, которые нам известны ($f1$ на картинке);
2. Использовать более простую функцию (в данном случае, линейную), которая не попадет точно во все данные, но зато будет соответствовать некоторым **общим закономерностям**, которые у них есть ($f2$ на картинке).

Характерной чертой переобучения является первый сценарий, и сопровождается он, как правило, **большими весами**. Поэтому регуляризацию используют для борьбы с переобучением.

<img src ="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L02/out/l2_regularization.png" width="300">

Проиллюстрируем описанное явление на примере полиномиальной модели:

In [ ]:
x = np.linspace(0, 2 * np.pi, 10)
y = np.sin(x) + np.random.normal(scale=0.25, size=len(x))
x_true = np.linspace(0, 2 * np.pi, 200)
y_true = np.sin(x_true)

plt.figure(figsize=(5, 3))
plt.scatter(x, y, s=50, facecolors="none", edgecolors="b", label="noisy data")
plt.plot(x_true, y_true, c="lime", label="ground truth")
plt.legend()
plt.show()

Попробуем аппроксимировать имеющуюся зависимость с помощью полиномиальной модели, используя шумные данные в качестве тренировочных данных:

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

x_train = x.reshape(-1, 1)

fig = plt.figure(figsize=(10, 5))

for i, degree in enumerate([0, 1, 3, 9]):
    model = make_pipeline(PolynomialFeatures(degree), LinearRegression())

    model.fit(x_train, y)
    y_plot = model.predict(x_true.reshape(-1, 1))

    fig.add_subplot(2, 2, i + 1)
    plt.plot(x_true, y_plot, c="red", label=f"M={degree}")
    plt.scatter(x, y, s=50, facecolors="none", edgecolors="b")
    plt.plot(x_true, y_true, c="lime")
    plt.legend()
plt.show()

Модель **переобучается, подстраиваясь под тренировочную выборку**. В полиноме степень/количество весов — это гиперпараметр, который можно подбирать на кросс-валидации, но **ограничивая количество параметров**, мы накладываем **ограничение на обобщающую способность модели** в целом. Вместо этого можно оставить модель сложной, но использовать **регуляризатор**, который будет заставлять модель отдавать предпочтение более простому обобщению.

## Регуляризация

Мы можем ограничить значения весов, введя в функционал ошибок
специальную добавку, называемую регуляризацией. L2-регуляризация имеет формулу:


$$\large L_2 = \alpha \sum_i w_i^2,$$
где $\alpha$ — это коэффициент регуляризации.


 Введение **L2-регуляризации** приводит к тому, что **большие веса больше штрафуются** и предпочтение отдается решениям, использующим **малые значения весов**. При этом модель будет **сохранять скоррелированные и неважные признаки с маленькими весами**.

Это связано с градиентом $L_2$:
$$\large L'_{2w_i} = 2\alpha w_i$$
Он будет “тянуть” модель в сторону большого количества маленьких весов.

<center><img src="https://edunet.kea.su/repo/EduNet-content/dev-2.1/L02/out/l1_and_l2_regularization.gif" alt="alttext" width="550"/></center>

Для отбора признаков можно использовать L1-регуляризацию, она одинаково "штрафует" модель за любые ненулевые веса.

$$\large L_1 = \alpha \sum_i |w_i|$$
$$\large L_{1w_i}' = \alpha, \text{  где } w_i\neq 0$$



Для получения интуиции, что L1-регуляризация позволяет отбирать признаки, обычно используют картинку ниже. На ней член функции потерь, отвечающий за регуляризацию, жестко ограничен: $|L_\text{reg}|≤1$. Вращающиеся овалы показывают, как Loss решаемой задачи изменяется в результате изменения входных данных и таргета.

Голубая область — ограничение на значения весов, которое дает регуляризация:
- Для **L2** это **окружность**.  Оранжевая точка — это минимальное значение для функции Loss с регуляризацией. Для **L2** она будет кататься **по касательной к окружности**.
- Для **L1** ограничения на значения весов будут иметь **форму ромба**. При этом минимальное значение для функции Loss с регуляризацией будет зависать в **уголу ромба**, что соответствует **обнулению веса** одного из признаков.



<center><img src ="https://edunet.kea.su/repo/EduNet-web_dependencies/dev-2.1/L02/loss_landscape_with_regularization.gif" width="800"></center>

<center><em>Source: <a href="https://people.eecs.berkeley.edu/~jrs/189/">Introduction to Machine Learning
</a></em></center>


In [ ]:
from sklearn.linear_model import Ridge


model = make_pipeline(PolynomialFeatures(9), LinearRegression())
model_ridge = make_pipeline(PolynomialFeatures(9), Ridge(alpha=0.1))

model.fit(x_train, y)
y_plot = model.predict(x_true.reshape(-1, 1))

model_ridge.fit(x_train, y)
y_plot_ridge = model_ridge.predict(x_true.reshape(-1, 1))

plt.figure(figsize=(5, 3))
plt.plot(x_true, y_plot, c="red", label=f"M={degree}")
plt.plot(x_true, y_plot_ridge, c="black", label=f"M={degree}, alpha=0.1")
plt.scatter(x, y, s=50, facecolors="none", edgecolors="b")
plt.plot(x_true, y_true, c="lime", label="ground truth")
plt.legend()
plt.show()

poly_coef = model[1].coef_

eq = f"y = {round(poly_coef[0], 2)}+{round(poly_coef[1], 2)}*x"
for i in range(2, 10):
    eq += f"+{round(poly_coef[i], 2)}*x^{i}"

print("Without regularization: ", eq)

poly_coef = model_ridge[1].coef_

eq = f"y = {round(poly_coef[0], 2)}+{round(poly_coef[1], 2)}*x"
for i in range(2, 10):
    eq += f"+{round(poly_coef[i], 2)}*x^{i}"

print("With regularization: ", eq)

Видно, что одним из "симптомов" переобучения являются аномально большие веса. Модель Ridge Regression, показанная в примере выше, использует L2-регуляризацию для борьбы с этим явлением.


<font size="6">Литература</font>

<font size="5">Инструменты:</font>

- [[doc] 🛠️ Scikit-learn](https://scikit-learn.org/stable/) — ML алгоритмы
- [[doc] 🛠️ NumPy](https://numpy.org/) — массивы и математические функции
- [[doc] 🛠️ Pandas](https://pandas.pydata.org/) — табличные данные
- [[doc] 🛠️ Matplotlib](https://matplotlib.org/) — визуализация
- [[doc] 🛠️ NLTK](https://www.nltk.org/) — токенизация, словари стоп-слов
- [[git] 🐾 RNNmorph](https://github.com/IlyaGusev/rnnmorph) — морфологический парсер для русского и английского языков
- [[doc] 🛠️ UDPipe](https://ufal.mff.cuni.cz/udpipe) — морфологический и синтаксический парсер для различных языков

<font size="5">Методы и алгоритмы:</font>

- [[wiki] 📚 Мешок слов](https://ru.wikipedia.org/wiki/Мешок_слов)
- [[wiki] 📚 TF-IDF](https://ru.wikipedia.org/wiki/TF-IDF)
- [[wiki] 📚 Наивный байесовский классификатор](https://ru.wikipedia.org/wiki/Наивный_байесовский_классификатор)
- [[wiki] 📚 Логистическая регрессия](https://ru.wikipedia.org/wiki/Логистическая_регрессия)

<font size="5">Подробные материалы:</font>
- [[wiki] 📚 MSU.AI Линейные модели](https://msu.ai/l02_linear_models)
- [[wiki] 📚 MSU.AI Деревья и леса](https://msu.ai/l03_classic_ml)

* [[book] 📚 Про разделение на классическое и глубокое машинное обучение ](https://www.deeplearningbook.org/contents/intro.html)
* [[book] 📚 Один из лучших учебников по классическому машинному обучению](https://web.stanford.edu/~hastie/ElemStatLearn/)
* [[book] 📚 Хорошая книга по машинному обучению](https://www.amazon.com/Hands-Machine-Learning-Scikit-Learn-TensorFlow/dp/1492032646)

<font size="5">Про деревья решений:</font>

* [[blog] ✏️ Деревья решений](https://www.kaggle.com/kashnitsky/topic-3-decision-trees-and-knn)

<font size="5">Про ансамбли:</font>

* [[blog] ✏️ Kaggle Ensembling guide](https://www.kaggle.com/code/amrmahmoud123/1-guide-to-ensembling-methods/notebook)
* [[blog] ✏️ Comprehensive guide for ensemble models](https://www.projectpro.io/article/a-comprehensive-guide-to-ensemble-learning-methods/432)
* [[blog] ✏️ Про стэкинг и блендинг](https://dyakonov.org/2017/03/10/c%D1%82%D0%B5%D0%BA%D0%B8%D0%BD%D0%B3-stacking-%D0%B8-%D0%B1%D0%BB%D0%B5%D0%BD%D0%B4%D0%B8%D0%BD%D0%B3-blending/)

<font size="5">XGBoost:</font>

* [[doc] 🛠️ Официальная документация](https://xgboost.readthedocs.io/en/stable/)
* [[blog] ✏️ Отличие XGBoost от обычного градиентного бустинга](https://stats.stackexchange.com/questions/202858/xgboost-loss-function-approximation-with-taylor-expansion)
* [[arxiv] 🎓 Оригинальная статья](https://arxiv.org/pdf/1603.02754.pdf)
* [[blog] ✏️ Как подбирать параметры XGBoost](https://usermanual.wiki/Document/Complete20Guide20to20Parameter20Tuning20in20XGBoost20with20codes20in20Python.1988513968)

<font size="5">CatBoost:</font>

* [[video] 📺 Решение задач классификации при помощи CatBoost](https://www.youtube.com/watch?v=xl1fwCza9C8)
* [[doc] 🛠️ Официальная документация](https://catboost.ai/en/docs/)

<font size="5">LightGBM:</font>

* [[doc] 🛠️ Очень подробная и удобная документация](https://lightgbm.readthedocs.io/en/latest/ )
* [[blog] ✏️ Описание параметров](https://neptune.ai/blog/lightgbm-parameters-guide?utm_source=datacamp&utm_medium=post&utm_campaign=blog-lightgbm-parameters-guide&utm_campaign=News&utm_medium=Community&utm_source=DataCamp.com)
* [[arxiv] 🎓 Новый бустинг с деревьями, содержащими в листах линейные регрессии](https://arxiv.org/pdf/1802.05640.pdf)

<font size="5">Дисбаланс классов:</font>

* [[blog] ✏️ Обучение в случае дисбаланса классов](http://www.svds.com/learning-imbalanced-classes/)
* [[blog] ✏️ Bagging и случайные леса для обучения с дисбалансом классов](https://machinelearningmastery.com/bagging-and-random-forest-for-imbalanced-classification/)
* [[article] 🎓 Коэффициент корреляции Мэтьюса](https://bmcgenomics.biomedcentral.com/articles/10.1186/s12864-019-6413-7)

<font size="5">Нейронные сети и бустинг:</font>

* [[arxiv] 🎓 TabNet](https://arxiv.org/pdf/1908.07442.pdf)
* [[git] 🐾 TabNet, реализация на PyTorch](https://github.com/dreamquark-ai/tabnet)
* [[arxiv] 🎓 NODE](https://arxiv.org/pdf/1909.06312.pdf)

